# Logic and Model-Based Reasoning

In [ ]:
from utils import *
from logic import *
from copy  import *

**Notes on Logic**

Syntax:

Logical **not** (negation): $\neg$, ~

Logical **and** (conjunction): $\wedge$, &

Logical **or** (disjunction): $\vee$, |

De Morgan’s Theorem

$\neg(A \wedge B) => (\neg A) \vee (\neg B)$

$\neg(A \vee B) => (\neg A) \wedge (\neg B)$

...

# Practice reducing sentances to CNF
The cell below will reduce logical sentances to CNF.  This is useful to check practice problems.  The following syntax is used:

Implies: ==>, >>

IFF: <=>

and: &

not: ~

or: |

The flag SHOW_STEPS controls how much of the process is displayed. 

SHOW_STEPS = False - Output is only final answer

SHOW_STEPS = True - Output shows each step in the reduction process

In [ ]:
#Examples
from logic import *
s = "A==>B"
to_cnf(s)

In [ ]:
s = "A<=>B"
to_cnf(s)

In [ ]:
s = "(A&B)|(C&D)"
to_cnf(s)

In [ ]:
s = "(A<=>B)&(B==>C)"
to_cnf(s, True)

In [ ]:
s = "(A<=>B)|~(D&C)"
to_cnf(s, True)

In [ ]:
#HW 9 Problems

In [ ]:
#B.1
SHOW_STEPS = True
s = "~((~A&B)|(C&D))"
to_cnf(s, SHOW_STEPS)

In [ ]:
#B.2
s = "A<=>A"
to_cnf(s, SHOW_STEPS)

In [ ]:
#B.3
s = "(A<=>B)|C"
to_cnf(s, SHOW_STEPS)

In [ ]:
SHOW_STEPS = True
s = "~((~A&B)|(C&D))"
to_cnf(s, SHOW_STEPS)

In [ ]:
#Demonstrate the steps of the unit propogation

In [ ]:
#Split a sentance into clauses
s = "~((~A&B)|(~C&D)|C|B)"
clauses = conjuncts(to_cnf(s))
to_cnf(s)

In [ ]:
#To find a valid model of the sentance, all clauses must be true
#Look for clauses with single values, and add their negations to the list of expressions that must be false
false_exp = []
true_exp = []
for c in clauses:
    if len(c.args)<=1:
        true_exp.append(c)
        false_exp.append(to_cnf(~c))
        print(false_exp)

In [ ]:
#Remove already satisfied clases
for i in true_exp:
    clauses.remove(i)

print clauses

In [ ]:
cp_clauses = deepcopy(clauses)
#Remove all literals known to be false from the other clases
for c in cp_clauses:
    for fe in false_exp:
        if fe in deepcopy(c.args):
            c.args.remove(fe)

print(cp_clauses)

#If any clauses are remaining with a single literal, add that literal to the known true/fault lists repeat
print(false_exp)
for c in cp_clauses:
    print(c)
    repeat = False
    if len(c.args) <= 1:
        if len(c.args) == 1:
            c =  expr(c.args[0].__repr__())
            print(c)
        true_exp.append(c)
        false_exp.append(to_cnf(~c))
        repeat = True

print(true_exp)
        


            

In [ ]:
def trim_or(clauses):
    #print type(clauses)
    #print "TRIMMING %s" % clauses

    clauses_to_trim = []
    for c in clauses:
        if len(c.args)==1 and (c.op == "|" or c.op == "|~"):
            clauses_to_trim.append(c)
    #print "ltt: %s"%len(clauses_to_trim)
    if len(clauses_to_trim) > 0:
        clauses = [c.args[0] if (c in clauses_to_trim) else c for c in clauses]
    #print "TRIMMED: %s"%clauses
    return clauses

def unit_propagation(s):
    #print type(s)
    #if type(s) == "<type 'str'>":
    s_cnf = to_cnf(s)
    clauses = conjuncts(s_cnf)
    #else:
    #    clauses = s
            
    repeat = True
    false_exp = []
    true_exp = []
    flag = True
    while (flag or repeat):
        #print "Loop Start"
        #print clauses
        if not repeat:
            flag = False
        #for c in clauses:
        #    if len(c.args)<=1:
        #        true_exp.append(c)
        #        false_exp.append(to_cnf(~c))
        repeat = False
        for c in clauses:
            #print len(clauses)
            #print "c: %s"%c
            if len(c.args) <= 1 and len(clauses) > 0:
                c2 = deepcopy(c)
                #print "c2:%s"%c2
                [c2] = trim_or([c2])
                #print "c2:%s"%c2
                if c not in true_exp:
                    #trim_or([c2])
                    true_exp.append(c2)
                    #print true_exp
                    false_exp.append(to_cnf(~c))
                    repeat = True 
                
        for e in true_exp:
            #print "e:%s"%e
            if e in clauses: clauses.remove(e) 
            #print "Clauses:%s"%clauses
            
        for c in clauses:
            for fe in false_exp:
                if fe in deepcopy(c.args):
                    #print ("Args:%s"%c.args)
                    #print ("Op:%s"%c.op)
                    if c.op != "~":
                        c.args.remove(fe)
                    #print "Clause after removing %s: %s"%(fe, clauses)
        clauses = trim_or(clauses)
    
    return clauses, true_exp

s = "(A1 ==> (X1 <=>(P1&V1)))&P1&~V1&A1"
print(unit_propagation(s))

In [ ]:
s = "~((~A&B)|(~C&D)|~C|B)"
print(to_cnf(s))

In [ ]:
#Boolean Spaceship Example
#Model that encodes the relationship between parts
C = []
C.append("A1 ==> (X1 <=>(P1&V1))")
C.append("A2 ==> (X2 <=>(P2&V2))")
C.append("A3 ==> (T1 <=>(X1&X2))")
C.append("A4 ==> (X3 <=>(V3&P1))")
C.append("A5 ==> (X4 <=>(V4&P2))")
C.append("A6 ==> (T2 <=>(X3&X4))")
C.append("A7 ==> (S1 <=>((T1&Y3)|(T2&Y3)))")
#C.append("R1 ==> (Y3 <=>((Y1|Y2)&(~Y1|~Y2)))")
C.append("R1 ==> (Y3 <=>((Y1|Y2)))")
C.append("C1 ==> (Y1 <=>B1)")
C.append("C2 ==> (Y2 <=>B2)")


def make_sentance(clauses):
    s = ""
    for c in clauses:
        repeat = False
        if len(c.args) <= 1:
            if len(c.args) == 1:
                c =  expr(c.args[0].__repr__())
            true_exp.append(c)
            false_exp.append(to_cnf(~c))
            repeat = True 
        
    return clauses, true_exp

s = "(A1 ==> (X1 <=>(P1&V1)))&P1&V1&A1"
print(unit_propagation(s))

In [ ]:
s = "~((~A&B)|(~C&D)|C|B)"
print(to_cnf(s))

In [ ]:
#Boolean Spaceship Example
#Model that encodes the relationship between parts
C = []
C.append("A1 ==> (X1 <=>(P1&V1))")
C.append("A2 ==> (X2 <=>(P2&V2))")
C.append("A3 ==> (T1 <=>(X1&X2))")
C.append("A4 ==> (X3 <=>(V3&P1))")
C.append("A5 ==> (X4 <=>(V4&P2))")
C.append("A6 ==> (T2 <=>(X3&X4))")
C.append("A7 ==> (S1 <=>((T1&Y3)|(T2&Y3)))")
C.append("R1 ==> (Y3 <=>((Y1&~Y2)|(~Y1&Y2)))")
#C.append("R1 ==> (Y3 <=>((Y1|Y2)))")
C.append("C1 ==> (Y1 <=>B1)")
C.append("C2 ==> (Y2 <=>B2)")


def make_sentance(clauses):
        s = ""
        for c in clauses:
            if s == "":
                s = "("+c+")"
            else:        
                s += "&("+c+")"
        return s

print(to_cnf(make_sentance(C)))

#All components functioning
C.append("A1")
C.append("A2")
C.append("A3")
C.append("A4")
C.append("A5")
C.append("A6")
C.append("A7")
C.append("R1")
C.append("C1")
C.append("C2")


print(unit_propagation(make_sentance(C)))


In [ ]:
#Assign a state to the system
#Propellant on
C.append("P1")
C.append("P2")

#Battery 1 on
C.append("B1")

#Battery 2 off
C.append("~B2")

#Valve 1 and 2 open
C.append("V1")
C.append("V2")

#Valve 2 and 3 closed
C.append("~V3")
C.append("~V4")

#print to_cnf(make_sentance(C))

unit_propagation(make_sentance(C))
